In [26]:
import os
import glob
import numpy as np
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import timecorr as tc
import math
import scipy.spatial.distance as sd
from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce
import mkl
mkl.set_num_threads(4)

factors = 700
if factors == 100:
    pieman_name = '../../data/pieman_ica100.mat'
else:
    pieman_name = '../../data/pieman_data.mat'



pieman_data = loadmat(pieman_name)  

pieman_conds = ['word']

data = []
conds = []


for c in pieman_conds:
    if c == 'paragraph':
        if factors == 700:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 3)[0]))
        else:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 0)[0]))
    else:
        next_data = list(map(lambda i: pieman_data[c][:, i][0], np.arange(pieman_data[c].shape[1])))
    data.extend(next_data)
    conds.extend([c]*len(next_data))


all_data = np.array(data)
conds = np.array(conds)
cfun = isfc
rfun = 'eigenvector_centrality'
width = 50
smooth = 'laplace'
raw = 'delta'

results_dir = '/dartfs/rc/lab/D/DBIC/CDL/f002s72/timecorr_paper/pieman/results/'

#results_dir = '/Users/lucyowen/Desktop/timecorr_env/timecorr_paper/pieman/results/'

corrsdir = os.path.join(results_dir, smooth + '_' + str(50) + '_orderedup_corrs')
if not os.path.exists(corrsdir):
    os.makedirs(corrsdir)
    

laplace = {'name': 'Laplace', 'weights': tc.laplace_weights, 'params': {'scale': width}}
delta = {'name': '$\delta$', 'weights': tc.eye_weights, 'params': tc.eye_params}
gaussian = {'name': 'Gaussian', 'weights': tc.gaussian_weights, 'params': {'var': width}}
mexican_hat = {'name': 'Mexican hat', 'weights': tc.mexican_hat_weights, 'params': {'sigma': width}}

smooth_parameter = eval(smooth)
raw_parameter = eval(raw)

smooth_fun=smooth_parameter['weights']
smooth_params=smooth_parameter['params']
raw_fun=raw_parameter['weights']
raw_params=raw_parameter['params']

combine = corrmean_combine  

levels = range(16)

for c in pieman_conds:
    for lev in levels:
        if os.path.exists(os.path.join(corrsdir, 'd_'+ str(lev) +'_'+ c +'.npy')):
            data = np.load(os.path.join(corrsdir, 'd_'+ str(lev) +'_'+ c +'.npy'))
            print(str(lev) + '_data loaded')
        else:
            if lev == 0:
                lev_data = all_data[conds == c]
            else:
                lev_data = data_r
            print('lev_data: ' + str(lev_data[0][0]))
            data = np.asarray(tc.timecorr([x for x in lev_data], cfun=isfc, rfun=None,
                                          weights_function=smooth_fun, weights_params=smooth_params))
            print('data: ' + str(data[0][0]))
            print(str(lev) + '_smooth data calculated')
            corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(data, axis=2)), axis=2))
            print(str(lev) + '_corrs calculated')
            np.save(os.path.join(corrsdir, 'lev_'+ str(lev) +'_'+ c +'.npy'), corr)
            data = np.asarray(tc.timecorr([x for x in lev_data], cfun=isfc, rfun=None,
                                          weights_function=raw_fun, weights_params=raw_params))
            print(str(lev) + '_raw data calculated')
            np.save(os.path.join(corrsdir, 'd_'+ str(lev) +'_'+ c +'.npy'), data)
        if os.path.exists(os.path.join(corrsdir, 'd_'+ str(lev) +'_r_'+ c +'.npy')):
            data_r = np.load(os.path.join(corrsdir, 'd_'+ str(lev) +'_r_'+ c +'.npy'))
            print(str(lev) + '_reduced data loaded')
        else:
            data_r = np.asarray(reduce([x for x in data], rfun=rfun))
            print(str(lev) + '_reduced data calculated')
            np.save(os.path.join(corrsdir, 'd_'+ str(lev) +'_r_'+ c +'.npy'), data_r)
            print('data_r: ' + str(data_r[0][0]))
        del data
        

range(0, 3)
0_data loaded
0_reduced data loaded
lev_data: [0.14302537 0.00982672 0.08145851 0.05860905 0.07720654 0.13107708
 0.08700501 0.12014306 0.02272707 0.12851244 0.09639177 0.13539257
 0.01057133 0.13790108 0.10585685 0.1308951  0.00367713 0.12067527
 0.09668716 0.12913936 0.128724   0.07483774 0.0746319  0.02725164
 0.08344455 0.12127144 0.08733651 0.14921866 0.02892407 0.12531941
 0.00920434 0.10757915 0.00811552 0.03756734 0.11342522 0.07662683
 0.13405174 0.05758245 0.02685659 0.12033433 0.06650679 0.05281363
 0.1424802  0.10382324 0.0140855  0.04675881 0.09704961 0.1055391
 0.05488077 0.13526464 0.02879268 0.00695268 0.08356988 0.02071912
 0.03402362 0.13897895 0.1128056  0.11573054 0.06335576 0.07580266
 0.13779102 0.1006125  0.13705344 0.12579201 0.04377354 0.11815308
 0.10788544 0.13513238 0.12071263 0.11163909 0.12464418 0.13629674
 0.09941703 0.13384385 0.07131796 0.12588853 0.12783232 0.0322023
 0.06038959 0.11707526 0.08449785 0.11743815 0.12465436 0.12224591
 0.121

## Cluster code to calculate correlations for each order - up to 4th order

In [3]:
import os
import glob
import numpy as np
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import timecorr as tc
import math
import scipy.spatial.distance as sd
from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce


In [4]:
data_dir = '/dartfs/rc/lab/D/DBIC/CDL/f002s72/timecorr_paper/pieman/results'
corrs_dir = os.path.join(data_dir, 'laplace_50_orderedup_corrs')

## For sliced correlations:

In [ ]:
#levels = np.arange(0,16,1)
levels = [12, 13, 14, 15]
#conditions = ['intact', 'paragraph', 'word', 'rest']
conditions = ['word']
n = .1
percentages = np.arange(n, 1 + n, n)
for l in levels:
    for c in conditions:
        con = os.path.join(corrs_dir, f'lev_{l}'+ f'_{c}' + '.npy')
        print(con)
        try:
            corrs = np.load(con)
            mat_corrs = tc.helpers.vec2mat(corrs)
            sliced_corrs = np.array([])
            next_corrdir = os.path.join(data_dir, 'sliced_corrs', f'level_{l}')
            if not os.path.exists(next_corrdir):
                os.makedirs(next_corrdir)
            for p in percentages:
                s = int(p*np.shape(mat_corrs)[-1])
                slice_corr = mat_corrs[:, :, s-1]
                if sliced_corrs.shape[0] == 0:
                    sliced_corrs = slice_corr
                else:
                    sliced_corrs = np.dstack((sliced_corrs, slice_corr))
            np.save(os.path.join(next_corrdir, f'{c}.npy'), sliced_corrs)
        except:
            print('issue loading: ' + con)
            pass

## For averaged correlations:

In [ ]:
# levels = np.arange(0,16,1)
# conditions = ['intact', 'paragraph', 'rest', 'word']

levels = [12, 13, 14, 15]
#conditions = ['intact', 'paragraph', 'word', 'rest']
conditions = ['word']

for l in levels:
    for c in conditions:
        con = os.path.join(corrs_dir, f'lev_{l}'+ f'_{c}'+ '.npy')
        try:
            corrs = np.load(con)
            mat_corrs = tc.helpers.vec2mat(corrs)
            next_corrdir = os.path.join(data_dir, 'mean_corrs', f'level_{l}')
            if not os.path.exists(next_corrdir):
                os.makedirs(next_corrdir)
            mean_corrs = mat_corrs.mean(axis=2)
            np.save(os.path.join(next_corrdir, f'{c}.npy'), mean_corrs)
        except:
            print('issue loading: ' + con)
            pass